# **RAG Application** with LangChain and HuggingFace LLM

In [ ]:
# Install the necessary packages
!pip install torch -q
!pip install transformers -q
!pip install numpy -q
!pip install langchain -q
!pip install langchain_community -q
!pip install langchain-chroma -q
!pip install sentence_transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.

In [ ]:
import os
from google.colab import userdata

### Initialize HuggingFace LLM

Model repo url: https://huggingface.co/mistralai/Mistral-7B-v0.1

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline
import torch

# Define the model ID
model_id = "mistralai/Mistral-7B-v0.1"

# Get your API token
huggingface_api_token = userdata.get('HUGGINGFACE_API_KEY')

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, token=huggingface_api_token)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16, token=huggingface_api_token)

# Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1, # Use temperature here as before
    trust_remote_code=True,
    device_map="auto",
)

# Initialize the HuggingFace llm using the pipeline
llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-3-6f7fe9f37a60>:4: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


### Initialize Embedding Model

Model url: https://sbert.net/

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
  model_name="sentence-transformers/all-mpnet-base-v2"
)

<ipython-input-4-89b57322def1>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python

### Initialize Output Parser

In [ ]:
from langchain.schema.output_parser import StrOutputParser

output_parser=StrOutputParser()

### Load PDF Document

In [ ]:
!pip install pypdf -qU

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("/content/codeprolk.pdf")

docs = loader.load()

In [ ]:
len(docs)

4

In [ ]:
docs[0]

Document(metadata={'source': '/content/codeprolk.pdf', 'page': 0}, page_content="Introduction to CodePRO LK  \nCodePRO LK  is a dynamic educational platform that offers a diverse range of technology -\nrelated courses in Sinhala, aimed at empowering Sri Lankans with valuable skills in \nprogramming, data science, and machine learning. Founded by Dinesh Piyasamara  during the \nCOVID -19 pandemic, CodePRO LK addresses the growing need for accessible, high -quality \ntech education tailored to the local community.  \n \nFounding and Vision  \nOrigin and Motivation  \nThe inception of CodePRO LK was driven by the challenges posed by the COVID -19 pandemic, \nwhich highlighted the necessity for remote learning and digital skills. Recognizing this, Dinesh \nPiyasamara launched CodePRO LK to provide Sri Lankan students with the  tools and knowledge \nto thrive in a digitally -driven world, all through their native language.  \nVision and Mission  \n• Vision : To assist talented Sri Lankans i

### Split Documents into Chunks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

20

### Create Vector Store and Retriever

In [ ]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [ ]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

### Define Prompt Template

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Define prompt template
template = """
Answer this question using the provided context only.

{question}

Context:
{context}

Answer:
"""

prompt=ChatPromptTemplate.from_template(template)

In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n\nAnswer:\n'), additional_kwargs={})])

### Chain Retriever and Prompt Template with LLM

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

#### Invoke RAG Chain with Example Questions

In [ ]:
response = chain.invoke("who is codeprolk?")
print(response)

Human: 
Answer this question using the provided context only.

who is codeprolk?

Context:
[Document(metadata={'page': 3, 'source': '/content/codeprolk.pdf'}, page_content='Partnerships and Collaborations  \nCodePRO LK is exploring partnerships with educational institutions, tech companies, and \nindustry experts to enrich its content and provide learners with access to a broader range of \nresources and opportunities. These collaborations aim to bridge the gap be tween education and \nindustry, ensuring that learners are well -prepared for real -world challenges.'), Document(metadata={'page': 1, 'source': '/content/codeprolk.pdf'}, page_content='Community and Support  \nCodePRO LK has cultivated a vibrant community where learners can interact, share insights, and \nsupport each other. Additionally, the platform offers consultation services for personalized \nlearning support.  \n \nCodePRO LK YouTube Channel  \nOverview  \nThe CodePRO LK YouTube Channel  is a crucial extension of the 

In [ ]:
response = chain.invoke("what are the courses they offer")
print(response)

Human: 
Answer this question using the provided context only.

what are the courses they offer

Context:
[Document(metadata={'page': 0, 'source': '/content/codeprolk.pdf'}, page_content='Course Offerings  \nVariety and Accessibility  \nCodePRO LK stands out for its wide array of free courses, all presented in Sinhala. The courses \ncater to various proficiency levels, from beginners to intermediates, ensuring that learners of all \nstages can benefit.  \nKey Courses  \n1. Python GUI – Tkinter : This course covers the essentials of creating graphical user'), Document(metadata={'page': 1, 'source': '/content/codeprolk.pdf'}, page_content='Learning Experience  \nCourse Structure  \nEach course is meticulously structured to provide a holistic learning experience, comprising:  \n• Video Lectures : Detailed tutorials that break down complex concepts.  \n• Quizzes : Interactive quizzes to reinforce learning.  \n• Assignments : Hands -on tasks to apply theoretical knowledge.  \nCommunity and S

In [ ]:
response = chain.invoke("what are the popular videos in codeprolk youtube channel")
print(response)

Human: 
Answer this question using the provided context only.

what are the popular videos in codeprolk youtube channel

Context:
[Document(metadata={'page': 2, 'source': '/content/codeprolk.pdf'}, page_content='Community Engagement  \nThe YouTube channel has amassed a substantial following, with thousands of subscribers \nactively engaging with the content. Viewers frequently leave comments expressing their \nappreciation and sharing how the videos have assisted them in their learning journ eys. \nImpact  \nThe CodePRO LK YouTube channel has played a significant role in democratizing tech'), Document(metadata={'page': 1, 'source': '/content/codeprolk.pdf'}, page_content='Community and Support  \nCodePRO LK has cultivated a vibrant community where learners can interact, share insights, and \nsupport each other. Additionally, the platform offers consultation services for personalized \nlearning support.  \n \nCodePRO LK YouTube Channel  \nOverview  \nThe CodePRO LK YouTube Channel  is a